In [1]:
# Dimensions
# lane -> Lane
# equipment_type -> Equipment Type
# carrier_name -> Carrier
# shipper_name -> Shipper

# Fact
# loadsmart_id -> Freigth

In [2]:
import pandas as pd

In [3]:
file_path = ''
file_name = '2017 Data- Challenge BI'
file_sheet = '2017 Data'

file_dim_lane = 'dim_lane'
file_dim_equipment_type = 'dim_equipment_type'
file_dim_carrier = 'dim_carrier'
file_dim_shipper = 'dim_shipper'
file_fact_loadsmart = 'fact_loadsmart'

In [4]:
def add_default_values(df_dim, column_name):
    return df_dim.append({'id': '-1', column_name:'Not Informed'}, ignore_index=True)

def get_fk(df_fact_table, df_dim, fk_name, column_drop):
    df_fact_table = df_fact_table.merge(df_dim, how='left')
    df_fact_table.rename({'id': fk_name}, axis='columns', inplace=True)
    df_fact_table.drop(columns=column_drop, inplace=True)
    
    return df_fact_table

def retrieve_dim_fk(df_fact):
    df_fact = get_fk(df_fact, df_dim_lane, 'lane_id', 'lane')
    df_fact = get_fk(df_fact, df_dim_equipment_type, 'equipment_type_id', 'equipment_type')
    df_fact = get_fk(df_fact, df_dim_carrier, 'carrier_id', 'carrier_name')
    df_fact = get_fk(df_fact, df_dim_shipper, 'shipper_id', 'shipper_name')
    
    df_fact.fillna(-1, inplace=True)
    
    return df_fact
 

def dim_type1_first_run(df_new):
    df_original = df_new.copy()
    df_original['id'] = df_original.index

    
    return df_original


def dim_type1_normal_run(df_original, df_new):
    df_original = df_original.append(df_new[~df_new.lane.isin(df_original.lane)], ignore_index = True)
    df_original['id'] = df_original.index
    
    return df_original

def fact_first_run(df_new):
    
    df_original = df_new.copy()
    df_original['loadsmart_key'] = (df_original.loadsmart_id.astype('str') + df_original.shipper_name.astype('str') + df_original.carrier_name.astype('str')).str.upper()    
    df_original = retrieve_dim_fk(df_original)
    
    return df_original

def fact_normal_run(df_original, df_new):
    df_new['loadsmart_key'] = (df_new.loadsmart_id.astype('str') + df_new.shipper_name.astype('str') + df_new.carrier_name.astype('str')).str.upper()
    df_to_append = df_new[~df_new.loadsmart_key.isin(df_original.loadsmart_key)], ignore_index = True    
    retrieve_dim_fk(df_to_append)
    
#   Append new records
    df_original = df_original.append(df_to_append)    
    
    
    return df_original

In [5]:
df_raw = pd.read_excel (file_path + file_name + '.xlsx', sheet_name= file_sheet)
df_raw.fillna(-1, inplace=True)

# Unique values
df_lane_new = pd.DataFrame({'lane':df_raw.lane.str.strip().unique()})
df_equipment_type_new = pd.DataFrame({'equipment_type':df_raw.equipment_type.str.strip().unique()})
df_carrier_new = pd.DataFrame({'carrier_name':df_raw.carrier_name.str.strip().unique()})
df_shipper_new = pd.DataFrame({'shipper_name':df_raw.shipper_name.str.strip().unique()})

In [6]:
# Dim Lane
try:
    df_dim_lane = pd.read_csv(file_path + file_dim_lane + '.csv', sep='|')
    df_dim_lane = dim_type1_normal_run(df_dim_lane, df_lane_new)
    df_dim_lane['pickup'] = df_dim_lane.lane.str.split('->').str[0].str.strip()
    df_dim_lane['delivery'] = df_dim_lane.lane.str.split('->').str[1].str.strip()
except:
    df_dim_lane = dim_type1_first_run(df_lane_new)
    df_dim_lane = add_default_values(df_dim_lane, 'lane')
    df_dim_lane['pickup'] = df_dim_lane.lane.str.split('->').str[0].str.strip()
    df_dim_lane['delivery'] = df_dim_lane.lane.str.split('->').str[1].str.strip()


# Dim Equipment Type
try:
    df_dim_equipment_type = pd.read_csv(file_path + file_dim_equipment_type + '.csv', sep='|')
    df_dim_equipment_type = dim_type1_normal_run(df_dim_equipment_type, df_equipment_type_new)
except:
    df_dim_equipment_type = dim_type1_first_run(df_equipment_type_new)
    df_dim_equipment_type = add_default_values(df_dim_equipment_type, 'equipment_type')
    
    
# Dim Carrier
try:
    df_dim_carrier = pd.read_csv(file_path + file_dim_carrier + '.csv', sep='|')
    df_dim_carrier = dim_type1_normal_run(df_dim_carrier, df_carrier_new)
except:
    df_dim_carrier = dim_type1_first_run(df_carrier_new)
    df_dim_carrier = add_default_values(df_dim_carrier, 'carrier_name')
    
    
# Dim Shipper
try:
    df_dim_shipper = pd.read_csv(file_path + file_dim_shipper + '.csv', sep='|')
    df_dim_shipper = dim_type1_normal_run(df_dim_shipper, df_shipper_new)
except:
    df_dim_shipper = dim_type1_first_run(df_shipper_new)
    df_dim_shipper = add_default_values(df_dim_shipper, 'shipper_name')
    
    
# Fact Loadsmart
try:
    df_fact_loadsmart = pd.read_csv(file_path + file_fact_loadsmart + '.csv', sep='|')
    df_fact_loadsmart = fact_normal_run(df_fact_loadsmart, df_raw)
except:
    df_fact_loadsmart = fact_first_run(df_raw)

In [7]:
df_dim_lane.to_csv(file_dim_lane + '.csv', sep='|', index=False)
df_dim_equipment_type.to_csv(file_dim_equipment_type + '.csv', sep='|', index=False)
df_dim_carrier.to_csv(file_dim_carrier + '.csv', sep='|', index=False)
df_dim_shipper.to_csv(file_dim_shipper + '.csv', sep='|', index=False)
df_fact_loadsmart.to_csv(file_fact_loadsmart + '.csv', sep='|', index=False)

In [8]:
# Assumptions:
# 1. No SCD Type 2 was created as the data did not have enough informartion for that
# 2. For the Fact table, no changes would be applied in a record once the data is loaded into BI (no reprocessing of old records)
# 3. Incremental load was considered for the Fact table
# 4. Carrier Rating and Vip Carrier could be loaded into Dim Carrier. For this exercise i kept in the fact as it could change along the time, but this would be a good candidadte for SCD Type 2
# 5. I noticed that has_mobile_app_tracking was duplicated, but I decided not to drop one of them as it would need further investigation
